In [1]:
import pandas as pd
import numpy as np
df_data=pd.read_excel(r"G:\Gayathri\Name Segmentation\Source\Keyword_Input1.xlsx")

print(df_data['New Type'].unique())

['Sweet Shop (Stand-alone)' 'Bakery (Stand-alone)' 'Chemist (Stand-alone)'
 'Kirana Str' 'Cig / Paan Kiosk' 'Juice Centr' 'Coffee Shop (Stand-alone)'
 'Restaurant' 'Hotel' 'Ice Cream Parlour (Stand-alone)'
 'Cosmetc / Fancy Str (Stand-alone)' 'Ayurvedic Pharma (Chain)'
 'Dairy str' 'Fruit / Veg Mandi' 'Tea Shop (Hot)' 'Fast Food'
 'Night Club / Bar' 'Canteen' 'Ayurvedic Pharma (Stand-alone)'
 'Chats & Snacks (Stand-alone)' 'Chemist (Chain)'
 'Grocry Dept Str (Stand-alone)' 'Gift Shop' 'Dry Fruit Str'
 'Grocry Dept Str (Chain)' 'Salon' 'Gourmet Str' 'Bakery (Chain)'
 'Chats & Snacks (Chain)' 'Cold Storage Str' 'Beauty Parlour'
 'Coffee Shop (Chain)' 'Rice / Flour Mill' 'Ice Cream Parlour (Chain)'
 'Cosmetc / Fancy Str (Chain)']


In [6]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622312 entries, 0 to 622311
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   RefID     622312 non-null  int64 
 1   CCP Name  622312 non-null  object
 2   Sub Type  622312 non-null  object
 3   New Type  622312 non-null  object
 4   Count     622312 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 23.7+ MB


In [5]:
df_data.head()

,RefID,CCP Name,Sub Type,New Type,Count
0,1,AGGARWAL SWEET CENTRE,Sweet Shop (Stand-alone),Sweet Shop (Stand-alone),31247
1,128,& JAY RAM TOURIST RASOI DHABA and FAST FOOD #J...,Sweet Shop (Stand-alone),Sweet Shop (Stand-alone),31247
2,170,(Chitranjan Hotel ) Shubham Sweets House,Sweet Shop (Stand-alone),Sweet Shop (Stand-alone),31247
3,185,(Gupta Sweet House),Sweet Shop (Stand-alone),Sweet Shop (Stand-alone),31247
4,187,(Hari Om Sweet Shop),Sweet Shop (Stand-alone),Sweet Shop (Stand-alone),31247


In [2]:
df_cnt=df_data.groupby('New Type').size().reset_index(name='Count')
df_data=pd.merge(df_data, df_cnt, on='New Type')

In [3]:
# df_data=df_data[df_data['Count']>=1000]

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# minority_classes = df_data[df_data['Count'] < 5000]
majority_classes = df_data[df_data['Count'] >= 10000]

# over_sampler = RandomOverSampler(sampling_strategy= {key: 5000 for key in minority_classes['New Type']})
under_sampler = RandomUnderSampler(sampling_strategy={key: 8000 for key in majority_classes['New Type']})

# X_over, y_over = over_sampler.fit_resample(np.array(minority_classes['CCP Name']).reshape(-1,1),np.array(minority_classes['New Type']).reshape(-1,1))
X_under, y_under = under_sampler.fit_resample(np.array(majority_classes['CCP Name']).reshape(-1,1),np.array(majority_classes['New Type']).reshape(-1,1))

# X=X_over.tolist()+X_under.tolist()
# y=y_over.tolist()+y_under.tolist()

X=X_under.tolist()+[[j] for j in df_data[df_data['Count'] < 5000]['CCP Name'].tolist()]
y=y_under.tolist()+df_data[df_data['Count'] < 5000]['New Type'].tolist()

balanced_df=pd.DataFrame({'CCP Name': [i[0] for i in X], 'New Type': y})

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import gensim
import re
from gensim.utils import simple_preprocess
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
corpus = []
for i in range(len(balanced_df['CCP Name'])):
    wrd = re.sub('[^a-zA-Z0-9]', ' ', str(balanced_df['CCP Name'][i]))
    words=wrd.split(' ')
    #words=simple_preprocess(wrd, deacc = False)
    words = [lemmatizer.lemmatize(word).lower().replace('  ','') for word in words if word.lower() not in set(stopwords.words('english'))]
    corpus.append(' '.join(words).strip().replace('  ',''))
    
balanced_df['New_Names']=corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sort\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sort\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sort\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sort\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(balanced_df['New Type'])

LabelEncoder()

In [8]:
balanced_df['Labels_Encode']=le.transform(balanced_df['New Type'])
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
X_count = count_vectorizer.fit_transform(balanced_df['CCP Name'])

In [9]:
from sklearn.decomposition import TruncatedSVD
tsd=TruncatedSVD(n_components=35)
X_count=tsd.fit_transform(X_count)

In [11]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_count, balanced_df['Labels_Encode'])


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [12]:
mp_test=pd.read_excel(r"G:\Gayathri\Name Segmentation\Kartikeyan_Source\Delhi\Delhi_Input.xlsx")

#%%

mp_corpus = []
for i in range(len(mp_test['Name'])):
    wrd = re.sub('[^a-zA-Z0-9]', ' ', str(mp_test['Name'][i]))
    words=wrd.split(' ')
    #words=simple_preprocess(wrd, deacc = False)
    words = [lemmatizer.lemmatize(word).lower().replace('  ','') for word in words if word.lower() not in set(stopwords.words('english'))]
    mp_corpus.append(' '.join(words).strip().replace('  ',''))

In [13]:

mp_test['New_Names']=mp_corpus

#%%

X_test_Count=count_vectorizer.transform(mp_test['New_Names'])
X_test_Count=tsd.transform(X_test_Count)

pred = model.predict(X_test_Count)
mp_test['Predicted_Type'] = le.inverse_transform(pred)


#%%

# print(classification_report(mp_test['Predicted_Type'], mp_test['My Sub Type']))
# mp_test.to_excel(r"G:\Gayathri\Name Segmentation\Check\Check1.xlsx")

#%%

mp_test['Predicted_Probability']=[max(i)*100 for i in model.predict_proba(X_test_Count)]

#%%

proba = model.predict_proba(X_test_Count)

mp_test['Predicted Class 1']=top_1_class=le.inverse_transform(proba.argsort()[:, ::-1][:, :1])
mp_test['Predicted Class 2']=top_2_class=le.inverse_transform(proba.argsort()[:, ::-1][:, 1:2])

#%%

# top_2_probabilities = np.sort(proba, axis=1)[:, ::-1][:, :2]
mp_test['Predicted Class 1 Prob']=top_1_class_prob=np.sort(proba, axis=1)[:, ::-1][:, :1]*100
mp_test['Predicted Class 2 Prob']=top_2_class_prob=np.sort(proba, axis=1)[:, ::-1][:, 1:2]*100



D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
mp_test.head()

,Refid,Category,Name,New_Names,Predicted_Type,Predicted_Probability,Predicted Class 1,Predicted Class 2,Predicted Class 1 Prob,Predicted Class 2 Prob
0,1,NaN,Agarwal Bikanar Sweets,agarwal bikanar sweets,Sweet Shop (Stand-alone),99.052852,Sweet Shop (Stand-alone),Bakery (Stand-alone),99.052849,0.460629
1,2,NaN,Delhi Darbar,delhi darbar,Restaurant,37.835297,Restaurant,Bakery (Stand-alone),37.835297,13.982844
2,3,NaN,Grocery Store,grocery store,Kirana Str,80.426455,Kirana Str,Grocry Dept Str (Stand-alone),80.426453,13.985062
3,4,NaN,Grocery Store,grocery store,Kirana Str,80.426455,Kirana Str,Grocry Dept Str (Stand-alone),80.426453,13.985062
4,5,NaN,M S Store,store,Kirana Str,39.668673,Kirana Str,Grocry Dept Str (Stand-alone),39.668674,27.588263


In [14]:

#mp_test.to_excel(r'G:\Gayathri\Name Segmentation\Kartikeyan_Source\Delhi\Delhi_Input_Segment_Added.xlsx')

